In [1]:
from bs4 import BeautifulSoup
import urllib.request as urllib2
from urllib.request import urlopen
import re
import csv
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import itertools

Schwab_BASE = 'https://www.schwabjobs.com'
PNC_BASE = 'https://www.indeed.com/viewjob?'

PNC_URL = "https://www.indeed.com/jobs?q=Pnc+Bank"
Schwab_URL = "https://www.schwabjobs.com/List/Custom/Schwab-Job-Category"

stateList = []

banks = ["PNC","Schwab"]
bank = ''
listId = 1
count = 1
BASE = 'https://www.indeed.com/'
FIRST_LISTING_PAGE = 'https://www.indeed.com/cmp/Pnc-Financial-Services-Group/jobs?clearPrefilter=1'
LISTING_URL_PREFIX = 'https://www.indeed.com/cmp/Pnc-Financial-Services-Group/jobs?start='
LISTING_URL_POSTFIX = 'clearPrefilter=1#cmp-menu-container'

def keywordsFileName(listId):
    if listId == 1:
        return 'frequency.csv'
    elif listId == 2:
        return 'tfidf.csv'
    else:
        return 'textrank.csv'

def scrape(postingUrl):
    print(postingUrl)
    global count
    headers = ["Job No", "Institution", "List Id {1,2,3}", "URL (URL of the job posting)", "List id (1,2,3)"]

    values = [count, bank, listId, postingUrl, listId]
    
    indices = list(range(1, 101))
    
    wordCountDict = dict.fromkeys(indices,0)
    postingDict = dict(zip(headers, values))
    
    enc = 'utf-8'
    with open(keywordsFileName(listId), 'r', encoding = enc) as f:
        reader = csv.reader(f)
        keywords = list(reader)
    arr = [i[0] for i in keywords]
    
    try:
        hdr = {'User-Agent': 'Mozilla/5.0'}
        request = urllib2.Request(postingUrl,headers=hdr)
        html_page = urlopen(request)
        soup2 =  BeautifulSoup(html_page,"html.parser")
        if bank == 'PNC Bank':
            value = soup2.find('input', {'id':'preLoadJSON'})
            for word in value.get('value').split('"CandLandPageText"')[1].replace("\\u003", " ").split():
                if(word in arr):
                    ind = arr.index(word)
                    wordCountDict[ind] = wordCountDict.get(ind, 0) + 1
            dataDict = postingDict.copy()  
            return {**postingDict, **wordCountDict}
                
        else:
            value = soup2.findAll("div", {"class": "desc"})
            for val in value:
                st = val.text.split()
                for word in st:
                    if(word in arr):
                        ind = arr.index(word)
                        wordCountDict[ind] = wordCountDict.get(ind, 0) + 1
    except:
        return ("An exception occurred")
    
    count = count + 1

    dataDict = postingDict.copy()  
    return {**postingDict, **wordCountDict} 
 
def scrape_page_pnc(page):
    response = urlopen(page)
    bs_obj = BeautifulSoup(response, "html.parser")
    if bs_obj:
        links = bs_obj.findAll('div',{'class': 'cmp-section cmp-with-border'})
        for page in links:
            linkTags = page.find_all('a', attrs={'href': re.compile("/cmp/_")})
            for tag in linkTags:
                link = tag.get('href')
                postingUrl = BASE + link
                pageDict = scrape(postingUrl)
                if (type(pageDict) is dict):
                        data = pd.DataFrame([pageDict])
                        # if file does not exist write header 
                        if not os.path.isfile('data.csv'):
                            data.to_csv('data.csv', header='column_names')
                        else: 
                            # else it exists so append without writing the header
                            data.to_csv('data.csv', mode='a', header=False)
                else:
                    print("exception?")
    


def scrape_pages_schwab(pages):
    stateList = {}
    for p in pages:
        pageHtml = urlopen(p)
        soup = BeautifulSoup(pageHtml)
        table = soup.find('table', 'JobListTable')
        pageDataframe = pd.DataFrame([])
        for link in table.findAll('a', attrs={'href': re.compile("/ShowJob")}):
            page = Schwab_BASE + link.get('href')
            state = table.findAll("td",{"class": "colstate"})
            for s in state:
                st = re.sub(r'[^ \w\.]', '', s.text)
                if st not in stateList:
                    stateList[st] = 1
                else:
                    stateList[st] += 1
            pageDict = scrape(page)
            if (type(pageDict) is dict):
                data = pd.DataFrame([pageDict])
                # if file does not exist write header 
                if not os.path.isfile('data.csv'):
                    data.to_csv('data.csv', header='column_names')
                else: # else it exists so append without writing the header
                    data.to_csv('data.csv', mode='a', header=False)
    with open('states.csv', 'a+', encoding = 'utf-8') as f: 
        for key in stateList.keys():
            f.write("%s,%d\n"%(key,stateList[key]))
        

def get_pages(categories):
    pageLinks = []
    for key, value in categories.items():
        if("---" not in key):
            if(value > 30):
                pageCount = 1
                for i in range(1,value,29):
                    pageLinks.append(Schwab_BASE + key + "Page-" + str(pageCount))
                    pageCount = pageCount + 1
                
            else:
                pageLinks.append(Schwab_BASE + key)
    return pageLinks

def get_category(response):
    bs_obj = BeautifulSoup(response, "html.parser")
    if bs_obj:
        cat_table = bs_obj.find_all("table",{"class": "list-categores-table"})

        catLinks = []
        numPages = []
        cat = []

        for page in cat_table:
            linkTags = page.find_all('a', attrs={'href': re.compile("/ListJobs")})
            divTags = page.find_all('div')
            for tag in linkTags:
                catLinks.append(tag.get('href'))
                cat.append(tag.text)
            for div in divTags:
                num = int(re.search(r'\d+', div.text).group())
                numPages.append(num)
        #categCsv = dict(zip(cat, numPages))
        #plt.bar(range(len(categCsv)), list(categCsv.values()), align='center')
        #plt.xticks(range(len(categCsv)), list(categCsv.keys()))
        #plt.imshow()
        return dict(zip(catLinks, numPages))

def scrape_site(Url):
    scrape_page_pnc(FIRST_LISTING_PAGE)
    for i in range(12,996,12):
        nextPageUrl = LISTING_URL_PREFIX +  str(i) + "&" + LISTING_URL_POSTFIX
        scrape_page_pnc(nextPageUrl)

def get_data ():
    if (bank == "PNC"):
        scrape_site(PNC_URL)
    elif (bank == "Schwab"):
        res = urllib2.urlopen(Schwab_URL)
        categories = get_category(res)
        pages = get_pages(categories)
        scrape_pages_schwab(pages)

if __name__ == '__main__':
    if os.path.isfile('data.csv'):
        os.remove('data.csv')
    for i in range(1,3):
        listId = i
        print(listId)
        for b in banks:
            bank = b
            print(bank)
            get_data()

1
PNC
https://www.indeed.com//cmp/_/job?jk=f5df4ed098b9df5e&tk=1d384pd110mct000
https://www.indeed.com//cmp/_/job?jk=cebe7bb3c395094e&tk=1d384pd110mct000
https://www.indeed.com//cmp/_/job?jk=33f0ec32a7975057&tk=1d384pd110mct000
https://www.indeed.com//cmp/_/job?jk=db3e14139dec1768&tk=1d384pd110mct000
https://www.indeed.com//cmp/_/job?jk=da5c29db7ae61182&tk=1d384pd110mct000
https://www.indeed.com//cmp/_/job?jk=91009db8062e49af&tk=1d384pd110mct000
https://www.indeed.com//cmp/_/job?jk=18b31206fcb52d2d&tk=1d384pd110mct000
https://www.indeed.com//cmp/_/job?jk=593da1a235fba740&tk=1d384pd110mct000
https://www.indeed.com//cmp/_/job?jk=5f0e0ffe66f32922&tk=1d384pd110mct000
https://www.indeed.com//cmp/_/job?jk=f4f1a766be0145e0&tk=1d384pd110mct000
https://www.indeed.com//cmp/_/job?jk=7ab091dc0cb157c5&tk=1d384pd110mct000
https://www.indeed.com//cmp/_/job?jk=d72b0df92538f6ef&tk=1d384pd110mct000
https://www.indeed.com//cmp/_/job?jk=2b989ad610c842ba&tk=1d384q22m0kbu000
https://www.indeed.com//cmp/_/jo

https://www.indeed.com//cmp/_/job?jk=2031cd243be903ac&tk=1d384v5ju0mte000
https://www.indeed.com//cmp/_/job?jk=14aaea12f5ac3308&tk=1d384v5ju0mte000
https://www.indeed.com//cmp/_/job?jk=2a335b13ce89ea7f&tk=1d384v5ju0mte000
https://www.indeed.com//cmp/_/job?jk=87868b88c50e143e&tk=1d384v5ju0mte000
https://www.indeed.com//cmp/_/job?jk=5f0e0ffe66f32922&tk=1d384v5ju0mte000
https://www.indeed.com//cmp/_/job?jk=f4f1a766be0145e0&tk=1d384v5ju0mte000
https://www.indeed.com//cmp/_/job?jk=8af9f505626fa11b&tk=1d384v5ju0mte000
https://www.indeed.com//cmp/_/job?jk=ac7e3c7219dc92a1&tk=1d384v5ju0mte000
https://www.indeed.com//cmp/_/job?jk=0190a55b1d53ce05&tk=1d384v5ju0mte000
https://www.indeed.com//cmp/_/job?jk=e931e15f14247627&tk=1d384v5ju0mte000
https://www.indeed.com//cmp/_/job?jk=9fb349c9e3755f29&tk=1d384vvhpak8i801
https://www.indeed.com//cmp/_/job?jk=8d46d4782ce80c80&tk=1d384vvhpak8i801
https://www.indeed.com//cmp/_/job?jk=16a205fc57c129eb&tk=1d384vvhpak8i801
https://www.indeed.com//cmp/_/job?jk=0

https://www.indeed.com//cmp/_/job?jk=c06c37921e1f83a3&tk=1d38538do0mct000
https://www.indeed.com//cmp/_/job?jk=b05356a72aa5ca6b&tk=1d38541iq0mte000&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKiKofo91uS6Q69ktC-y-BiMHNS3ifTU3X4Uxq_eODsJbhaKG6vxZlsYClqA2a8pSwxaDhLrCoZvIAVuhL8RNTx9XG9Bu8BKBUikC6K5gZyTt57hLmMtjcy0AOAevDEivqIXJNiMwFn8pn2sbxVvbrixeSpT1VFqEKZfaU_ve9HltNu5kDfwVFrBpl958_bJe05dNx7_iWtBcE6snd6aumweS9Q0y8SjUuKIAJJrEko4yNBMnYq78gj3yjdMbCqjp5nnX9IfN12uaE6t-yPb8rVbQOQ-j_DZU0mvT-Xe859lgHMPD976BkZQGzCFFe2ScnT8R6AECFbQIiFLPtr_IG19wOhfZmdKbl32vNHMH_9vlz-OEiiVhRbjfFwBx5MA0YkLKlbMIvQeiiFXGun_Xpl8HTUDcX6MW1Hyl0uSK3hyd9SH9ohU0Yo90l1gxYCCd15h_tShg60g8JvLfPTQ13Q8nP8Ke8WzOjUP6v_roAQUCAoscty1Z0o4QKKv5X3rUJ-rJflvZ56qFNB3KXYyAVHvgdSWo2EM6Az069lUHzIfbeaTFiXBddPUunqCHnbs4a8JpCkg
https://www.indeed.com//cmp/_/job?jk=4ad1171bbca9123e&tk=1d38541iq0mte000
https://www.indeed.com//cmp/_/job?jk=00b2aac911359c2f&tk=1d38541iq0mte000&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKiKofo91uS6Q69ktC-y-BiMHNS3ifTU3X4Uxq_eODs

https://www.indeed.com//cmp/_/job?jk=4b9748c3af300f5e&tk=1d3857por0kbu000
https://www.indeed.com//cmp/_/job?jk=6545f9db89782d03&tk=1d3857por0kbu000
https://www.indeed.com//cmp/_/job?jk=f1d93398ebdf9a61&tk=1d3857por0kbu000
https://www.indeed.com//cmp/_/job?jk=2f1e747c13cdd927&tk=1d3857por0kbu000
https://www.indeed.com//cmp/_/job?jk=c1d23a5616fa6d61&tk=1d3857por0kbu000
https://www.indeed.com//cmp/_/job?jk=bd3c60b6bf6eae46&tk=1d3857por0kbu000
https://www.indeed.com//cmp/_/job?jk=435b73db7189c1a0&tk=1d3857por0kbu000
https://www.indeed.com//cmp/_/job?jk=6808573258418763&tk=1d3857por0kbu000
https://www.indeed.com//cmp/_/job?jk=86a071a747d58814&tk=1d3857por0kbu000
https://www.indeed.com//cmp/_/job?jk=d104cc73650920c5&tk=1d3857por0kbu000&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKiKofo91uS6Q69ktC-y-BiMHNS3ifTU3X4Uxq_eODsJbhaKG6pWY1xKe9vZtztShvwGdXacff-gWxt6UVtbm70F25D3tJxrFoyAfusTMl8ZlTqHBTFkoKBqEJgI_Y9Yek3bEvcOazS11mGCnndJbvs5rli8pMLQqhpzU7Z3D-8bkU9WIcX1ykmPLOJ1aLxiMt5oR9di_DG0z2Pszvxeh93HAmzNhm

https://www.indeed.com//cmp/_/job?jk=ee4f9da921b8e6f5&tk=1d385br8n0mct000
https://www.indeed.com//cmp/_/job?jk=be4fdd4fbb08d1e9&tk=1d385br8n0mct000
https://www.indeed.com//cmp/_/job?jk=20a5ca87ff72706c&tk=1d385br8n0mct000
https://www.indeed.com//cmp/_/job?jk=391e60eccc4694cb&tk=1d385br8n0mct000
https://www.indeed.com//cmp/_/job?jk=4f2cc8dd0a11d0ea&tk=1d385br8n0mct000
https://www.indeed.com//cmp/_/job?jk=56bda605490bcf7d&tk=1d385br8n0mct000
https://www.indeed.com//cmp/_/job?jk=c64c760583c74dbc&tk=1d385br8n0mct000
https://www.indeed.com//cmp/_/job?jk=6dae4042a33937a0&tk=1d385br8n0mct000
https://www.indeed.com//cmp/_/job?jk=0f3db8177c089d25&tk=1d385br8n0mct000
https://www.indeed.com//cmp/_/job?jk=b3e4481fec54ab3f&tk=1d385br8n0mct000
https://www.indeed.com//cmp/_/job?jk=6304ef6f838fa9ea&tk=1d385br8n0mct000
https://www.indeed.com//cmp/_/job?jk=1fc1cc992d494b48&tk=1d385cjf80mtv000
https://www.indeed.com//cmp/_/job?jk=11b9966e0c4654b1&tk=1d385cjf80mtv000
https://www.indeed.com//cmp/_/job?jk=8

...

https://www.schwabjobs.com/ShowJob/Id/1832389/Sr.-Manager,-QA-Engineering/
https://www.schwabjobs.com/ShowJob/Id/1942675/Senior-Manager-–-Big-Data-System-Support-Engineer/
https://www.schwabjobs.com/ShowJob/Id/1942476/Senior-Staff,-Enterprise-Architecture/
https://www.schwabjobs.com/ShowJob/Id/1871700/MOS-System-Platform-Owner-–-Client-Verification-and-Risk-Engine/
https://www.schwabjobs.com/ShowJob/Id/1856627/Managing-Director-Data-Platforms-Investment-Management/
https://www.schwabjobs.com/ShowJob/Id/1909070/Sr-Manager-Software-Development/
https://www.schwabjobs.com/ShowJob/Id/1909915/Managing-Director,-IT-Project-Management-AST/
https://www.schwabjobs.com/ShowJob/Id/1615129/Managing-Director,-Test-Automation-Investment-Management/
https://www.schwabjobs.com/ShowJob/Id/1941616/Middle-Office-Services-(MOS)-Senior-Systems-Business-Analyst/
https://www.schwabjobs.com/ShowJob/Id/1907429/Build-Release-Engineer/
https://www.schwabjobs.com/ShowJob/Id/1855586/Associate-Salesforce-Developer/

https://www.schwabjobs.com/ShowJob/Id/1833891/Senior-Staff-Software-Applications-Engineer/
https://www.schwabjobs.com/ShowJob/Id/1839248/Sr.-Staff,-Scrum-Master/
https://www.schwabjobs.com/ShowJob/Id/1835529/Sr.-Staff,-Software-Support-Engineer/
https://www.schwabjobs.com/ShowJob/Id/1869511/Sr-Staff-DevOps-Engineer/
https://www.schwabjobs.com/ShowJob/Id/1834048/Staff-Software-Engineer/
https://www.schwabjobs.com/ShowJob/Id/1872271/Sr.-Staff,-DevOps-Tools-Engineer/
https://www.schwabjobs.com/ShowJob/Id/1761042/Solutions-Architect,-Data-Management-Data-Governance/
https://www.schwabjobs.com/ShowJob/Id/1783142/Technical-Director-Application-Security-Architect/
https://www.schwabjobs.com/ShowJob/Id/1860527/Staff-C#-.NET-Developer-(ERN4)/
https://www.schwabjobs.com/ShowJob/Id/1864589/Sr.-Staff-Security-Engineer/
https://www.schwabjobs.com/ShowJob/Id/1710623/Staff,-Java-Engineer/
https://www.schwabjobs.com/ShowJob/Id/1533454/Platform-Technical-Director/
https://www.schwabjobs.com/ShowJob/Id/

https://www.schwabjobs.com/ShowJob/Id/1930201/Associate-Creative-Director-Copywriter/
https://www.schwabjobs.com/ShowJob/Id/1934493/Sr.-Field-Marketing-Manager/
https://www.schwabjobs.com/ShowJob/Id/1939860/Print-Production-Manager/
https://www.schwabjobs.com/ShowJob/Id/1938458/Sr.-Digital-Marketing-Manager/
https://www.schwabjobs.com/ShowJob/Id/1934240/UX-Architect/
https://www.schwabjobs.com/ShowJob/Id/1947624/Sr.-Product-Manager,-Digital-Platform-Solutions/
https://www.schwabjobs.com/ShowJob/Id/1751372/Sr.-Product-Manager,-Sales-Accelerator/
https://www.schwabjobs.com/ShowJob/Id/1595226/Director,-Fixed-Income-Solutions/
https://www.schwabjobs.com/ShowJob/Id/1841559/Managing-Director-–-MF-ETF-Platform-Client-Experience/
https://www.schwabjobs.com/ShowJob/Id/1871493/Digital-Program-Manager/
https://www.schwabjobs.com/ShowJob/Id/1939458/Business-Technology-Process-Specialist/
https://www.schwabjobs.com/ShowJob/Id/1896504/Senior-Project-Manager-Change-Management-Lead/
https://www.schwab

https://www.schwabjobs.com/ShowJob/Id/1905388/Stock-Plan-Services-Financial-Services-Professional/
https://www.schwabjobs.com/ShowJob/Id/1903906/Stock-Plan-Services-Representative-eServices/
https://www.schwabjobs.com/ShowJob/Id/1898487/Account-Executive-Stock-Plan-Services-Administration/
https://www.schwabjobs.com/ShowJob/Id/1908553/Stock-Plan-Services-Financial-Services-Professional/
https://www.schwabjobs.com/ShowJob/Id/1905288/Financial-Reporting-Analyst-Stock-Plan-Services/
https://www.schwabjobs.com/ShowJob/Id/1901080/Managing-Director,-SPS-Strategy-and-Participant-Experience/
https://www.schwabjobs.com/ShowJob/Id/1898513/Stock-Plan-Services-Client-Service-Administrator/
https://www.schwabjobs.com/ShowJob/Id/1898524/Stock-Plan-Services-Client-Service-Administrator/
2
PNC
https://www.indeed.com//cmp/_/job?jk=f5df4ed098b9df5e&tk=1d3879cr00mte000


FileNotFoundError: [Errno 2] No such file or directory: 'tfidf.csv'